In [1]:
import os
import wfdb
import numpy as np
import math
import sys
import scipy.stats as st
from sklearn.preprocessing import scale
import glob, os
from os.path import basename
from keras import regularizers

In [2]:
# load data
perct=0.81 #percentage training
percv=0.19 #percentage validation
exclude = set()
#exclude.update(["234"])# no P annotated:
#datfiles=glob.glob(qtdbpath+"*.dat")

In [3]:
#import numpy as np
import tensorflow as tf
from keras.layers import Dense,Activation,Dropout
from keras.layers import LSTM,Bidirectional, GRU #could try TimeDistributed(Dense(...))
from keras.models import Sequential, load_model
from keras import optimizers,regularizers
from keras.layers.normalization import BatchNormalization
import tensorflow.compat.v1.keras.backend as KTF
from keras.metrics import categorical_accuracy
from keras.callbacks import EarlyStopping

np.random.seed(42)




# functions
def get_ecg_data(datfile): 
    ## convert .dat/q1c to numpy arrays
    recordname=os.path.basename(datfile).split(".dat")[0]
    recordpath=os.path.dirname(datfile)
    print(recordname,recordpath)
#     cwd=os.getcwd()
#     os.chdir(recordpath) ## somehow it only works if you chdir. 

    annotator='atr'
    annotation = wfdb.rdann(recordname, extension=annotator, sampfrom=0,sampto = None)
    
    record = wfdb.rdrecord(recordname, sampfrom=0,sampto = None) #wfdb.showanncodes()
    
    Vctrecord=np.transpose(record.p_signal)
    
    VctAnnotationHot=np.zeros( (2,len(Vctrecord[1])), dtype=np.int)
    VctAnnotationHot[1] = 1;
    
    VctAnnotations=list(zip(annotation.sample,annotation.symbol)) ## zip coordinates + annotations (N),(t) etc)
    #print(VctAnnotations)
    for i in range(len(VctAnnotations)):
        if(  VctAnnotations[i][1] == '·' or 
           VctAnnotations[i][1] == 'N' or 
            VctAnnotations[i][1] == 'L' or  
            VctAnnotations[i][1] == 'R' or 
           VctAnnotations[i][1] == 'B' or 
           VctAnnotations[i][1] == 'A' or
           VctAnnotations[i][1] == 'a' or 
           VctAnnotations[i][1] == 'J' or 
           VctAnnotations[i][1] == 'S' or 
           VctAnnotations[i][1] == 'V' or  
           VctAnnotations[i][1] == 'r' or  
           VctAnnotations[i][1] == 'F' or 
           VctAnnotations[i][1] == 'e' or 
           VctAnnotations[i][1] == 'j' or
           VctAnnotations[i][1] == 'n' or 
           VctAnnotations[i][1] == 'E' or 
           VctAnnotations[i][1] == '/' or 
           VctAnnotations[i][1] == 'f' or
           VctAnnotations[i][1] == 'Q' or  
           VctAnnotations[i][1] == '?'):
            VctAnnotationHot[0][VctAnnotations[i][0]] = 1;  
            VctAnnotationHot[1][VctAnnotations[i][0]] = 0;  
    VctAnnotationHot=np.transpose(VctAnnotationHot)
    Vctrecord=np.transpose(Vctrecord) # transpose to (timesteps,feat)

#     os.chdir(cwd)
    return Vctrecord, VctAnnotationHot


In [4]:
def splitseq(x,n,o):
	n = int(n)
	o = int(o)
	#split seq; should be optimized so that remove_seq_gaps is not needed. 
	upper= int(math.ceil( x.shape[0] / n) *n)
	print("splitting on",n,"with overlap of ",o,	"total datapoints:",x.shape[0],"; upper:",upper)
	for i in range(0,upper,n):
		#print(i)
		if i==0:
			padded=np.zeros( ( o+n+o,x.shape[1])   ) ## pad with 0's on init
			padded[o:,:x.shape[1]] = x[i:i+n+o,:]
			xpart=padded
		else:
			xpart=x[i-o:i+n+o,:]
		if xpart.shape[0]<i:

			padded=np.zeros( (o+n+o,xpart.shape[1])  ) ## pad with 0's on end of seq
			padded[:xpart.shape[0],:xpart.shape[1]] = xpart
			xpart=padded

		xpart=np.expand_dims(xpart,0)## add one dimension; so that you get shape (samples,timesteps,features)
		try:
			xx=np.vstack(  (xx,xpart) )
		except UnboundLocalError: ## on init
			xx=xpart
	print("output: ",xx.shape)
	return(xx)

In [5]:
def normalize_new(x):
    for i in range(x.shape[0]):
        x[i] = scale( x[i], axis=0, with_mean=True, with_std=True, copy=True )
    return x

In [6]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


In [7]:
def LoaddDatFiles(datfiles):  
    for datfile in datfiles:
        print(datfile)
        if basename(datfile).split(".",1)[0] in exclude:
            continue
        qf=os.path.splitext(datfile)[0]+'.atr'
        if os.path.isfile(qf):
#             print("yes",qf,datfile)
            x,y=get_ecg_data(datfile)

            x,y=splitseq(x,1000,0),splitseq(y,1000,0) ## create equal sized numpy arrays of n size and overlap of o 

            x = normalize_new(x)
            ## todo; add noise, shuffle leads etc. ?
            try: ## concat
                xx=np.vstack(  (xx,x) )
                yy=np.vstack(  (yy,y) )
            except NameError: ## if xx does not exist yet (on init)
                xx = x
                yy = y
    return(xx,yy)

In [8]:
## MITDB


#qtdbpath_test="/home/federico/imec-nl/ECG-LSTM-Annotator/physionet/physiobank/database/mitdb/"#sys.argv[1] ## first argument = qtdb database from physionet. 

# load data
import glob
datfiles_test=glob.glob("*.dat")
datfiles_test


xxtmitdb,yytmitdb=LoaddDatFiles(datfiles_test) # training data. 
xxtmitdb,yytmitdb=unison_shuffled_copies(xxtmitdb,yytmitdb) ### shuffle


# USE SINGLE CHANNEL
pcha=xxtmitdb[:,:,1]; 
pchb=xxtmitdb[:,:,0];

all_x_test = np.vstack([pchb]) # only channel zero
all_x_test = np.expand_dims(all_x_test,2)
all_lab_test = np.vstack([1-yytmitdb]) # only channel zero
print(all_lab_test.shape)
print(all_x_test.shape)
print("!!!!!!!!!!!!!!!")

16273.dat
16273 
splitting on 1000 with overlap of  0 total datapoints: 11354112 ; upper: 11355000
output:  (11355, 1000, 2)
splitting on 1000 with overlap of  0 total datapoints: 11354112 ; upper: 11355000
output:  (11355, 1000, 2)
(11355, 1000, 2)
(11355, 1000, 1)
!!!!!!!!!!!!!!!


In [9]:
seqlength=xxtmitdb.shape[1]
features=1#xxtmitdb.shape[2]
dimout=yytmitdb.shape[2]
print("xxv/validation shape: {}, Seqlength: {}, Features: {}".format(xxtmitdb.shape[0],seqlength,features))

xxv/validation shape: 11355, Seqlength: 1000, Features: 1


In [10]:
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.compat.v1.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

# def f1(y_true, y_pred):
#     y_pred = K.round(y_pred)
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

#     p = tp / (tp + fp + K.epsilon())
#     r = tp / (tp + fn + K.epsilon())

#     f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#     return K.mean(f1)


In [11]:
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from tensorflow.keras import layers
from keras import optimizers
# from tensorflow.python.keras.models import Sequential,load_model

import numpy as np
from keras.activations import softmax
from keras.objectives import categorical_crossentropy

def getmodel_one():
    model = Sequential()
    model.add(Dense(32, kernel_regularizer=l2(l=0.01), input_shape=(seqlength, features)))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))#, input_shape=(seqlength, features)) ) ### bidirectional ---><---
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(l=0.01)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(dimout, activation='sigmoid'))
    adam = tf.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    weights = np.array([64.0, 0.1,])
    model.compile(loss=f1_loss, #loss=weighted_categorical_crossentropy(weights), 
                  optimizer=adam, 
                  metrics=['categorical_accuracy', f1_m,precision_m, recall_m, matthews_correlation]) #(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy']) 
    print(model.summary())
    return(model)

In [12]:
def get_session(gpu_fraction=0.8):
	#allocate % of gpu memory.
	num_threads = os.environ.get('OMP_NUM_THREADS')
	gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
	if num_threads:
		return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
	else:
		return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [13]:
## USE CPU otherwise need a lot of memory in the GPU
from keras import backend as K
import tensorflow as tf
num_cores = 4
GPU = 1
CPU = 0

if GPU:
    num_GPU = 1
    num_CPU = 6
if CPU:
    num_CPU = 7
    num_GPU = 0

config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.compat.v1.Session(config=config)
KTF.set_session(session)

In [14]:
# call keras/tensorflow and build lstm model 
#index_ = np.linspace(0, xxt.shape[0]-1, xxt.shape[0] ).astype(int);
#shuffle(index_)

# TRAIN
KTF.set_session(session)

## early stopping monitoring mette
EarlyStopping(monitor='matthews_correlation', mode='max')

with tf.compat.v1.Session(): #switch to /cpu:0 to use cpu 
    #if not os.path.isfile('model_mitd_binary_noover.h5'):
    model =   getmodel_one() #build_model_gru() 
    #model = getmodel_simple();
    for this_eps in range(8):
        model.fit(all_x_test, all_lab_test, batch_size=100, epochs=1, verbose=1) # train the model
        model.save('model_edb_binary_noover_eps_f1loss_2ch_'+str(this_eps)+'_.h5')
        model.save_weights('model_weights_edb_binary_noover_eps_f1loss_2ch_'+str(this_eps)+'_.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000, 32)          64        
_________________________________________________________________
bidirectional (Bidirectional (None, 1000, 128)         49664     
_________________________________________________________________
dropout (Dropout)            (None, 1000, 128)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1000, 128)         512       
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 32)          4128      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000, 32)          1

In [15]:
#load weights because of the custom metrics 
from keras.models import load_model

with tf.device('/cpu:0'): #switch to /cpu:0 to use cpu 
   
    model = getmodel_one()
    model.load_weights('model_weights_edb_binary_noover_eps_f1loss_2ch_1_.h5')
 
    loss, cat_acc , f1_score, precision, recall, mettc = model.evaluate(all_x_test, all_lab_test, batch_size=100, verbose=1)
    print('f1_score: {}'.format(f1_score))
    print('CatAcc: {}'.format(cat_acc))
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print('MattCorr: {}'.format(mettc))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1000, 32)          64        
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 128)         49664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1000, 128)         512       
_________________________________________________________________
dense_4 (Dense)              (None, 1000, 32)          4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 32)          0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 1000, 32)         